In [1]:
import pandas as pd
import requests
from scipy.stats import ttest_ind

In [2]:
DATA_VERSION = "v2"

merged_data_path = f'../data/{DATA_VERSION}/merged_data.jsonl'
data = pd.read_json(merged_data_path, lines=True)

In [3]:
data = data.sample(frac=0.1)

In [ ]:
group_a_predictions = []
group_b_predictions = []

requests.delete("https://ium.rasztabiga.me/ab_test/results")

for index, row in data.iterrows():
    user_id = row ['user_id']
    genres = row['genres']
    favourite_genres = row['favourite_genres']
    skipped = row['skipped']

    group = user_id % 2

    prediction = requests.post(f"https://ium.rasztabiga.me/ab_test?user_id={user_id}", json={"genres": genres, "favourite_genres": favourite_genres}).json()

    if group == 0:
        group_a_predictions.append({"prediction": prediction["skipped"], "skipped": skipped})
    else:
        group_b_predictions.append({"prediction": prediction["skipped"], "skipped": skipped})

In [ ]:
group_a_accuracy = []
for prediction in group_a_predictions:
    group_a_accuracy.append(int(prediction["prediction"] == prediction["skipped"]))

group_b_accuracy = []
for prediction in group_b_predictions:
    group_b_accuracy.append(int(prediction["prediction"] == prediction["skipped"]))

group_a_accuracy_percentage = sum(group_a_accuracy) / len(group_a_accuracy)
group_b_accuracy_percentage = sum(group_b_accuracy) / len(group_b_accuracy)

print(f"Group A accuracy: {group_a_accuracy_percentage}")
print(f"Group B accuracy: {group_b_accuracy_percentage}")

Group A accuracy: 0.7616438356164383
Group B accuracy: 0.6709470304975923


In [ ]:
print(len(group_a_predictions))
print(len(group_b_predictions))

365
623


Hipoteza zerowa: dokładność modelu A nie jest większa niż modelu B

Hipoteza alternatywna: dokładność modelu A jest większa niż modelu B

Zastosuj statystykę t-Studenta
Zakładamy, że współczynnik wystąpienia błędu typu pierwszego wynosi alfa = 0.05

Odrzucimy hipotezę zerową, pod warunkiem, że p-value < alpha

In [ ]:
# Ustalamy poziom istotności
alpha = 0.05

# Przeprowadzamy test t-Studenta
t_statistic, p_value = ttest_ind(group_a_accuracy, group_b_accuracy, alternative="greater")

print("Wartość t-statystyki:", t_statistic)
print("Wartość p-value:", p_value)

# Sprawdzamy, czy odrzucamy hipotezę zerową
if p_value < alpha:
    print("Odrzucamy hipotezę zerową. Dokładność modelu A jest większa niż modelu B.")
else:
    print("Nie możemy odrzucić hipotezy zerowej. Nie ma wystarczających dowodów na to, że dokładność modelu A jest większa niż modelu B.")


Wartość t-statystyki: 3.026444516084243
Wartość p-value: 0.001269366320691488
Odrzucamy hipotezę zerową. Dokładność modelu A jest większa niż modelu B.
